In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from catboost import Pool, CatBoostClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv('train.csv')
label_encoder = LabelEncoder()

# Drop column
train = train.drop(["id"], axis=1)

# fill Na
train.fillna(0.0, inplace=True)

# encoded column:
encoded_columns = ["Name", "City", "Gender", "Age", "Working Professional or Student", "Profession", "Sleep Duration", "Dietary Habits", "Degree", "Have you ever had suicidal thoughts ?", "Family History of Mental Illness"]

for column in encoded_columns:
    train[column] = train[column].astype(str)
    train[column] = label_encoder.fit_transform(train[column])

train = train.astype(float)
print(len(train))
train.head(5)

140700


,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,11.0,0.0,31.0,50.0,1.0,11.0,0.0,5.0,0.00,0.0,2.0,29.0,8.0,34.0,0.0,1.0,2.0,0.0,0.0
1,407.0,1.0,8.0,93.0,1.0,56.0,0.0,4.0,0.00,0.0,3.0,27.0,21.0,64.0,1.0,7.0,3.0,0.0,1.0
2,417.0,1.0,15.0,97.0,0.0,0.0,5.0,0.0,8.97,2.0,0.0,15.0,8.0,22.0,1.0,3.0,1.0,0.0,1.0
3,417.0,1.0,4.0,64.0,1.0,56.0,0.0,5.0,0.00,0.0,1.0,27.0,16.0,29.0,1.0,10.0,1.0,1.0,1.0
4,286.0,0.0,12.0,37.0,1.0,10.0,0.0,1.0,0.00,0.0,1.0,15.0,21.0,29.0,1.0,9.0,4.0,1.0,0.0


In [5]:
X = train.drop(columns=["Depression"])
y = train["Depression"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [7]:
random_forest_model = RandomForestClassifier(random_state=42, n_estimators=100) 
random_forest_model.fit(X_train, y_train)

y_pred = random_forest_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9317697228144989
Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96     11527
         1.0       0.83      0.78      0.81      2543

    accuracy                           0.93     14070
   macro avg       0.89      0.87      0.88     14070
weighted avg       0.93      0.93      0.93     14070

Confusion Matrix:
[[11120   407]
 [  553  1990]]


In [8]:
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [16]:
cv = StratifiedKFold(5, shuffle=True, random_state=0)
cv_splits = cv.split(X, y)
cv_splits

<generator object _BaseKFold.split at 0x00000204AC0AB520>

In [17]:
for i, (train_idx, val_idx) in enumerate(cv_splits):
    print(i, len(train_idx), len(val_idx))

0 112560 28140
1 112560 28140
2 112560 28140
3 112560 28140
4 112560 28140
